In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Swish activation
def swish(x):
    return x * tf.nn.sigmoid(x)

# Squeeze-and-Excitation block
def se_block(inputs, se_ratio=0.25):
    input_channels = inputs.shape[-1]
    reduced_channels = max(1, int(input_channels * se_ratio))

    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1, 1, input_channels))(x)
    x = layers.Conv2D(reduced_channels, kernel_size=1, activation='swish')(x)
    x = layers.Conv2D(input_channels, kernel_size=1, activation='sigmoid')(x)
    return layers.Multiply()([inputs, x])

# MBConv block
def mbconv_block(inputs, expansion_factor, output_channels, kernel_size, stride, se_ratio, drop_rate):
    input_channels = inputs.shape[-1]
    expanded_channels = input_channels * expansion_factor

    # Expansion phase
    if expansion_factor != 1:
        x = layers.Conv2D(expanded_channels, kernel_size=1, padding='same', use_bias=False)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(swish)(x)
    else:
        x = inputs

    # Depthwise Convolution
    x = layers.DepthwiseConv2D(kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Squeeze-and-Excitation
    x = se_block(x, se_ratio)

    # Projection phase
    x = layers.Conv2D(output_channels, kernel_size=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Residual connection
    if stride == 1 and input_channels == output_channels:
        if drop_rate > 0.1:
            x = layers.Dropout(drop_rate)(x)
        x = layers.Add()([inputs, x])
    return x

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# ✅ Custom Lag Compensation Layer
class LagCompensationLayer(layers.Layer):
    def __init__(self, K=1.0, T1=0.1, **kwargs):
        super(LagCompensationLayer, self).__init__(**kwargs)
        self.K = K
        self.T1 = T1

    def call(self, inputs):
        return self.K * (inputs + self.T1 * tf.nn.relu(inputs))

# ✅ Custom Lead Compensation Layer
class LeadCompensationLayer(layers.Layer):
    def __init__(self, K=1.0, T2=0.05, **kwargs):
        super(LeadCompensationLayer, self).__init__(**kwargs)
        self.K = K
        self.T2 = T2

    def call(self, inputs):
        return self.K * (inputs - self.T2 * tf.nn.relu(-inputs))

# ✅ Build EfficientNet-B3 with Lag-Lead Compensation
def build_efficientnet_b3_with_compensation(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Load EfficientNet-B3 as feature extractor
    base_model = tf.keras.applications.EfficientNetB3(include_top=False, input_tensor=inputs, weights='imagenet')
    x = base_model.output

    # Apply Lag Compensation as a Layer
    x_lag = LagCompensationLayer()(x)

    # Apply Lead Compensation as a Layer
    x_lead = LeadCompensationLayer()(x)

    # Merge compensated outputs
    x = layers.Concatenate()([x, x_lag, x_lead])  # Increase feature depth

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Dense Output Layer (Binary Classification)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

# ✅ Define Input Shape (300x300 RGB)
input_shape = (300, 300, 3)

# ✅ Build Model
model = build_efficientnet_b3_with_compensation(input_shape)

# ✅ Display Model Summary
model.summary()